# Track

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import sys
import os

import numpy as np

import unittest

## Main Class

In [2]:
class Track():
    '''Basic GPS track class'''

    def __init__(self):
        '''Basic init'''

        self.name = None
        
        self.data = {}
        self.fields = {}

        self.numPoints = 0


    def summarise(self):
        '''Summarise the tracks after loading'''
        
        # Count the number of records
        self.numPoints = 0

        for fieldName, array in self.data.items():
            if self.numPoints and array.size != self.numPoints:
                raise RuntimeError('Inconsistent array length - field {}'.format(fieldName))
            else:
                self.fields[fieldName] = self.getFieldInfo(array)
                self.numPoints = array.size


    def getFieldInfo(self, array):
        '''Determine the number of decimal places being used by the field'''

        return {'type': array.dtype.name, 'decimals': self.getDecimals(array)}


    def getDecimals(self, array, maxDecDigits=None):
        '''Determine the number of decimal places being used in the array'''

        # maxDecDigits is currently unused but is available for the future
        if maxDecDigits is None:
            maxAbs = np.max(np.abs(array))
            if maxAbs > 0:
                maxIntDigits = int(np.floor(np.log10(maxAbs)) + 1)
            else:
                maxIntDigits = 0

            if array.dtype.name == 'float64':
                maxDecDigits = 15 - maxIntDigits
            elif array.dtype.name == 'float32':
                maxDecDigits = 6 - maxIntDigits
            elif array.dtype.name == 'float16':
                maxDecDigits = 3 - maxIntDigits
            else:
                return 0

        # Scale the float so that it can be treated as an integer
        scale = 10 ** maxDecDigits
        scaled = np.round(array * scale)

        divisor = 10
        decimals = maxDecDigits

        while divisor <= scale and not np.any(scaled % divisor):
            divisor *= 10
            decimals -= 1

        return decimals

## Unit Tests

In [3]:
class TestReaders(unittest.TestCase):
    '''Class to test all of the file readers'''

    def testFloat32Dec2(self):
        '''Test float32 - 2 decimals'''

        track = Track()
        
        array = np.array([0.12, 6543.21], dtype='float32')
        
        arrayInfo = track.getFieldInfo(array)
        
        self.assertEqual(arrayInfo['type'], 'float32')
        self.assertEqual(arrayInfo['decimals'], 2)


    def testFloat32Dec3(self):
        '''Test float32 - 3 decimals'''

        track = Track()
        
        array = np.array([.123, 654.321], dtype='float32')
        
        arrayInfo = track.getFieldInfo(array)
        
        self.assertEqual(arrayInfo['type'], 'float32')
        self.assertEqual(arrayInfo['decimals'], 3)


    def testFloat64Dec2(self):
        '''Test float64 - 2 decimals'''

        track = Track()
        
        array = np.array([0.12, 9876543.21], dtype='float64')
        
        arrayInfo = track.getFieldInfo(array)
        
        self.assertEqual(arrayInfo['type'], 'float64')
        self.assertEqual(arrayInfo['decimals'], 2)


    def testFloat64Dec3(self):
        '''Test float64 - 3 decimals'''

        track = Track()
        
        array = np.array([.123, 987654.321], dtype='float64')
        
        arrayInfo = track.getFieldInfo(array)
        
        self.assertEqual(arrayInfo['type'], 'float64')
        self.assertEqual(arrayInfo['decimals'], 3)

In [4]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

....
----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK
